In [2]:
import pandas as pd
from pandas import DataFrame
import sklearn
from sklearn.cluster import KMeans
import matplotlib
%matplotlib inline

In [3]:
train_path = './57_hypothyroid/TRAIN/dataset_TRAIN/tables/learningData.csv'
test_path = './57_hypothyroid/TEST/dataset_TEST/tables/learningData.csv'

In [4]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [4]:
print(train_data.columns)

Index(['d3mIndex', 'age', 'sex', 'on_thyroxine', 'query_on_thyroxine',
       'on_antithyroid_medication', 'sick', 'pregnant', 'thyroid_surgery',
       'I131_treatment', 'query_hypothyroid', 'query_hyperthyroid', 'lithium',
       'goitre', 'tumor', 'hypopituitary', 'psych', 'TSH_measured', 'TSH',
       'T3_measured', 'T3', 'TT4_measured', 'TT4', 'T4U_measured', 'T4U',
       'FTI_measured', 'FTI', 'TBG_measured', 'TBG', 'referral_source',
       'Class'],
      dtype='object')


In [4]:
#train_data = train_data.drop('d3mIndex', axis=1)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3018 entries, 0 to 3017
Data columns (total 30 columns):
age                          3017 non-null float64
sex                          2897 non-null object
on_thyroxine                 3018 non-null object
query_on_thyroxine           3018 non-null object
on_antithyroid_medication    3018 non-null object
sick                         3018 non-null object
pregnant                     3018 non-null object
thyroid_surgery              3018 non-null object
I131_treatment               3018 non-null object
query_hypothyroid            3018 non-null object
query_hyperthyroid           3018 non-null object
lithium                      3018 non-null object
goitre                       3018 non-null object
tumor                        3018 non-null object
hypopituitary                3018 non-null object
psych                        3018 non-null object
TSH_measured                 3018 non-null object
TSH                          2714 non-null

In [5]:
train_data = train_data.dropna(axis=1, how='all')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3018 entries, 0 to 3017
Data columns (total 30 columns):
d3mIndex                     3018 non-null int64
age                          3017 non-null float64
sex                          2897 non-null object
on_thyroxine                 3018 non-null object
query_on_thyroxine           3018 non-null object
on_antithyroid_medication    3018 non-null object
sick                         3018 non-null object
pregnant                     3018 non-null object
thyroid_surgery              3018 non-null object
I131_treatment               3018 non-null object
query_hypothyroid            3018 non-null object
query_hyperthyroid           3018 non-null object
lithium                      3018 non-null object
goitre                       3018 non-null object
tumor                        3018 non-null object
hypopituitary                3018 non-null object
psych                        3018 non-null object
TSH_measured                 3018 non-null 

2098 rows out of 3018 contain no NA values.

In [6]:
print(train_data.cov())

            age         TSH         T3          TT4       T4U          FTI  \
age  419.823776  -22.318305  -4.039062   -27.989082 -0.661944    36.092942   
TSH  -22.318305  550.246690  -3.076187  -222.325519  0.296704  -238.569204   
T3    -4.039062   -3.076187   0.694689    17.062767  0.077400    10.113822   
TT4  -27.989082 -222.325519  17.062767  1285.059166  3.177610   946.738278   
T4U   -0.661944    0.296704   0.077400     3.177610  0.039244    -1.094783   
FTI   36.092942 -238.569204  10.113822   946.738278 -1.094783  1103.487607   
TBG         NaN         NaN        NaN          NaN       NaN          NaN   

     TBG  
age  NaN  
TSH  NaN  
T3   NaN  
TT4  NaN  
T4U  NaN  
FTI  NaN  
TBG  NaN  


In [19]:
train_data.describe()

,d3mIndex,age,TSH,T3,TT4,T4U,FTI
count,3018.000000,3017.000000,2714.000000,2394.000000,2827.000000,2696.000000,2696.000000
mean,1883.002319,51.904541,4.962017,2.019173,108.496852,0.996218,110.468064
std,1085.121051,20.489602,23.457338,0.833480,35.847722,0.198100,33.218784
min,0.000000,1.000000,0.005000,0.100000,2.000000,0.360000,2.000000
25%,949.250000,36.000000,0.500000,1.600000,87.000000,0.870000,93.000000
50%,1891.500000,54.000000,1.400000,2.000000,104.000000,0.970000,107.000000
75%,2812.750000,68.000000,2.700000,2.400000,125.000000,1.080000,124.000000
max,3771.000000,455.000000,530.000000,10.600000,430.000000,2.320000,395.000000


Something isn't right with the ages. There's a max age of 455 (likely 45). Only one though, so that's easy to correct.

In [27]:
age = train_data['age']
print(age.sort_values(ascending=True))

2190      1.0
2127      1.0
2044      1.0
181       1.0
90        1.0
613       1.0
1250      2.0
1350      2.0
1111      2.0
973       4.0
1414      6.0
2895      7.0
332       7.0
1203      7.0
1035      8.0
2197      8.0
222      10.0
2315     11.0
2530     11.0
1745     11.0
998      11.0
2455     12.0
958      12.0
987      13.0
1301     13.0
2039     13.0
691      13.0
754      14.0
2755     14.0
2206     14.0
        ...  
2026     87.0
618      88.0
2107     88.0
1607     88.0
2813     88.0
1926     88.0
2799     88.0
1196     88.0
2839     88.0
2189     89.0
2588     89.0
1029     89.0
299      89.0
1702     89.0
1636     89.0
2299     89.0
353      90.0
1117     90.0
1245     90.0
450      90.0
1795     90.0
1380     91.0
1189     91.0
2217     92.0
2254     92.0
1929     93.0
2436     93.0
2148     94.0
1083    455.0
1586      NaN
Name: age, Length: 3018, dtype: float64


In [39]:
float_df = train_data.select_dtypes(include=[float]).dropna(axis=0, how='any')
kmeans_model = KMeans(n_clusters=4)
kmeans_model = kmeans_model.fit(float_df)
print(kmeans_model.labels_)

[1 0 1 ... 1 0 0]


In [ ]:
# plot kmeans clusters
# perform PCA analysis